In [10]:
import torch
import torch.nn as nn #all neural network
import torch.optim as optim #optimizers
import torch.nn.functional as F #activation functions
from torch.utils.data import DataLoader #minibatch and data functions
import torchvision.datasets as datasets #standard datasets
import torchvision.transforms as transforms #transformations on datasets

In [57]:
from tqdm import tqdm  # For nice progress bar!

In [2]:
#create a basic network

In [17]:
class NN(nn.Module):
    def __init__(self,input_size, num_classes): #784 inputsize
        super(NN,self).__init__()#init the inti class
        self.fc1 = nn.Linear(input_size,50) #layer1 of inputsize
        self.fc2 = nn.Linear(50, num_classes) #layer2 hidden layer 50 nodes
    
    def forward(self,x):
        x =F.relu(self.fc1(x)) #fc1 on x and put relu on it
        x= self.fc2(x)#
        return x
        

In [20]:
# explain of shape and working of model
model= NN(784,10)#784= no of neurons and 10 is number of digits
x= torch.randn(64,784) #64 is no of samples of digits in 784 neurons
model(x).shape


torch.Size([64, 10])

In [21]:
# explain layers of model
nn.Linear(784, 10) #layer 1

Linear(in_features=784, out_features=10, bias=True)

In [24]:
# explain layers of model
nn.Linear(50, 10) #layer2

Linear(in_features=50, out_features=10, bias=True)

In [27]:
#set device
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [29]:
#hyperparameters
input_size = 784
num_classes =10
learning_rate =0.001
batch_size =64
num_epochs =1

In [34]:
#loaddata
train_dataset=datasets.MNIST(root ="dataset/", train=True, transform=transforms.ToTensor(),\
              download =True)


In [42]:
train_loader = DataLoader(dataset= train_dataset, batch_size=batch_size, shuffle=True)
train_loader

In [41]:
test_dataset = datasets.MNIST(root ="dataset/", train=False , transform=transforms.ToTensor())
test_dataset

Dataset MNIST
    Number of datapoints: 10000
    Root location: dataset/
    Split: Test
    StandardTransform
Transform: ToTensor()

In [45]:
test_loader= DataLoader(dataset=test_dataset,batch_size=batch_size, shuffle=True)
test_loader

In [46]:
NN(input_size=input_size, num_classes=num_classes).to(device)

NN(
  (fc1): Linear(in_features=784, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [47]:
model=NN(input_size=input_size, num_classes=num_classes).to(device)

In [53]:
#Loss
criterion= nn.CrossEntropyLoss()
criterion

CrossEntropyLoss()

In [56]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    maximize: False
    weight_decay: 0
)

In [86]:
k=data.reshape(data.shape,-1)
k.shape

torch.Size([32, 1, 28, 28])

In [87]:
for epoch in range(num_epochs):
    for batch_idx,( data,targets) in enumerate(tqdm(train_loader)): \
        #get data and traget in batches
        
        #get data in cuda
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        #get correct shape of data for model
        data =data.reshape(data.shape[0],-1)  #torch.Size([32, 1, 28, 28]) to one
        
        #Forward pass 
        scores = model(data)
        loss = criterion(scores,targets)
        
        #Backward
        optimizer.zero_grad() #initilizes gradient to 0 before calculating back pass
        loss.backward()
        
        #Gradient descent or adam step
        optimizer.step()
        
        

100%|██████████| 938/938 [00:11<00:00, 82.83it/s] 


In [93]:
def check_accuracy(loader,model):
    num_correct=0
    num_samples=0
    model.eval()
    
    with torch.no_grad():
        #move eval dat to device
        for x, y in loader :
            x=x.to(device=device)
            y=y.to(device=device)

            #get correct shape
            x =x.reshape(x.shape[0],-1)

            #forward pass
            scores =model(x)
            _, predictions =scores.max(1)

            #check how many we got correct
            num_correct += (predictions ==y ).sum()

            #keep track of num of samples
            num_samples += predictions.size(0)

    model.train()
    return num_correct /num_samples #calculate accuracy


        

In [94]:
print(check_accuracy(train_loader, model)*100)

tensor(93.4167)


In [95]:
print(check_accuracy(test_loader, model)*100)

tensor(93.5000)
